# 2.1 Prompt Caching Implementation

**Learning Objectives**:
- Implement prompt caching with boto3 Converse API and Invoke Model API
- Apply caching strategy best practices for optimal performance
- Measure cache hit rates and cost savings

---

## Setup

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import json
import boto3
from datetime import datetime

# Verify AWS credentials
if os.getenv('AWS_ACCESS_KEY_ID') and os.getenv('AWS_SECRET_ACCESS_KEY'):
    print("✅ AWS credentials loaded from .env")
    print(f"✅ Region: {os.getenv('AWS_DEFAULT_REGION', 'us-east-1')}")
else:
    print("✅ Using AWS CLI credentials")
    print("ℹ️  boto3 will use credentials from ~/.aws/credentials")

# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=os.getenv('AWS_DEFAULT_REGION', 'us-east-1')
)

print("\n✅ Bedrock client initialized")

# Model configuration (use global.anthropic for cross-region inference)
MODEL_ID = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"

✅ Using AWS CLI credentials
ℹ️  boto3 will use credentials from ~/.aws/credentials

✅ Bedrock client initialized


## Load Helper Functions

We'll use cache metrics utilities from the `utils` module.

In [2]:
# Import cache metrics utilities
from utils.cache_metrics import (
    extract_cache_metrics,
    print_cache_metrics,
    calculate_cache_savings
)

print("✅ Helper functions loaded from utils module")

✅ Helper functions loaded from utils module


---

# Part A: Prompt Caching Implementation

In this part, you'll learn:

1. **Using Prompt Caching with Converse and Invoke Model APIs** - Learn how to add cache checkpoints with both API styles and understand their differences
2. **Multi-Checkpoint Caching** - Cache multiple layers with different update frequencies (tools, system, conversation history)
3. **Cache Invalidation** - Understand how cache behaves when content changes

---

## Section 1: Prompt Caching with Converse and Invoke Model APIs

Amazon Bedrock supports two APIs for calling foundation models. Both provide **identical caching functionality** - only the syntax differs.

| Aspect | **Converse API** (Recommended) | **Invoke Model API** |
|--------|-------------------------------|---------------------|
| **Purpose** | Unified interface across all Bedrock models | Direct access to provider-specific APIs |
| **Abstraction** | High-level, handles provider details automatically | Low-level, uses native provider format |
| **Benefits** | Consistent syntax across all providers (Anthropic, Amazon, Meta, etc.) | Full control over provider-specific features |
| **Cache Syntax** | `{"cachePoint": {"type": "default"}}` | `{"cache_control": {"type": "ephemeral"}}` |
| **Use Case** | Modern applications, easier to switch models | Provider-specific capabilities, SDK compatibility |

**In this workshop**: We'll first demonstrate both approaches below to show the syntax differences, then use Converse API throughout (recommended for new implementations).

---

### Implementation: Simple Prompt Caching

We'll implement the document Q&A system with caching using **both** Converse and Invoke APIs to highlight their differences.

**You'll see:**
- How to structure the cache checkpoint with each API
- The syntax differences between `cachePoint` (Converse) and `cache_control` (Invoke)  
- Both APIs produce identical caching behavior

In [3]:
# Load product documentation from file
with open('data/product_manual.txt', 'r') as f:
    PRODUCT_MANUAL = f.read()

# Note: Each cache checkpoint must meet minimum token requirements
# Learn more: https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-caching.html

print(f"✅ Product manual loaded")
print(f"   Length: {len(PRODUCT_MANUAL.split())} words")
print(f"   Estimated tokens: ~{len(PRODUCT_MANUAL.split()) * 1.3:.0f} tokens")

✅ Product manual loaded
   Length: 1023 words
   Estimated tokens: ~1330 tokens


### Implementation Option 1: Converse API

In [4]:
def query_document_converse(user_query, document=PRODUCT_MANUAL, model_id=MODEL_ID):
    """
    Query a document using single-checkpoint caching with Converse API.
    
    Args:
        user_query: User's question
        document: Static document to cache
        model_id: Bedrock model ID
    
    Returns:
        tuple: (response_text, cache_metrics)
    """
    # Construct message with cache checkpoint
    messages = [
        {
            "role": "user",
            "content": [
                {
                    # Static content to cache
                    "text": f"""You are a helpful assistant. Use the following product manual to answer questions.

PRODUCT MANUAL:
{document}

Answer the user's question based on the information in the manual. If the answer is not in the manual, say so."""
                },
                {
                    # CACHE CHECKPOINT (Converse API - separate block)
                    "cachePoint": {"type": "default"}
                },
                {
                    # User query (dynamic, not cached)
                    "text": f"\n\nQUESTION: {user_query}"
                }
            ]
        }
    ]
    
    # Call Bedrock Converse API
    response = bedrock_runtime.converse(
        modelId=model_id,
        messages=messages,
        inferenceConfig={
            "maxTokens": 500,
            "temperature": 0.0
        }
    )
    
    # Extract response and metrics
    response_text = response['output']['message']['content'][0]['text']
    metrics = extract_cache_metrics(response)
    
    return response_text, metrics

print("✅ Converse API function defined")

✅ Converse API function defined


### Implementation Option 2: Invoke Model API

In [5]:
def query_document_invoke(user_query, document=PRODUCT_MANUAL, model_id=MODEL_ID):
    """
    Query a document using single-checkpoint caching with Invoke Model API.
    
    Args:
        user_query: User's question
        document: Static document to cache
        model_id: Bedrock model ID
    
    Returns:
        tuple: (response_text, cache_metrics)
    """
    # Construct request body with cache_control syntax (Invoke Model API)
    # Add "Invoke Model API" marker to create separate cache from Converse demo
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 500,
        "temperature": 0.0,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"""You are a helpful assistant (Invoke Model API). Use the following product manual to answer questions.

PRODUCT MANUAL:
{document}

Answer the user's question based on the information in the manual. If the answer is not in the manual, say so.""",
                        # CACHE CHECKPOINT (Invoke Model API syntax)
                        "cache_control": {"type": "ephemeral"}
                    },
                    {
                        "type": "text",
                        # User query (dynamic, not cached)
                        "text": f"\n\nQUESTION: {user_query}"
                    }
                ]
            }
        ]
    }
    
    # Call Bedrock Invoke Model API
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body)
    )
    
    # Parse response
    response_body = json.loads(response['body'].read())
    
    # Extract response text
    response_text = response_body['content'][0]['text']
    
    # Extract metrics (Invoke Model API format)
    usage = response_body.get('usage', {})
    metrics = {
        'input_tokens': usage.get('input_tokens', 0),
        'output_tokens': usage.get('output_tokens', 0),
        'cache_write': usage.get('cache_creation_input_tokens', 0),  # Different key name
        'cache_read': usage.get('cache_read_input_tokens', 0)        # Different key name
    }
    
    return response_text, metrics

print("✅ Invoke Model API function defined")

✅ Invoke Model API function defined


### Demo: Converse API Example

Let's test the caching with multiple queries using the Converse API.

In [6]:
# Test queries
queries = [
    "What is the return policy for electronics?",
    "How much does the Professional tier cost?",
    "What shipping options are available?",
    "What are the API rate limits for standard tier?"
]

all_metrics_converse = []

print("\n" + "="*80)
print("DEMO: Single-Checkpoint Caching with Converse API")
print("="*80)

for i, query in enumerate(queries, 1):
    print(f"\n🔍 Query {i}: {query}")
    
    response, metrics = query_document_converse(query)
    all_metrics_converse.append(metrics)
    
    print(f"\n💬 Response: {response[:100]}..." if len(response) > 100 else f"\n💬 Response: {response}")
    print_cache_metrics(metrics, request_num=i)

# Calculate savings
print("\n" + "="*80)
print("COST ANALYSIS - Converse API")
print("="*80)

savings = calculate_cache_savings(all_metrics_converse)
print(f"\nTotal requests: {savings['total_requests']}")
print(f"Cache hit rate: {savings['cache_hit_rate']:.1f}%")
print(f"\nCost with caching:    ${savings['cost_with_cache']:.6f}")
print(f"Cost without caching: ${savings['cost_no_cache']:.6f}")
print(f"\n💰 Savings: ${savings['savings']:.6f} ({savings['savings_pct']:.1f}% reduction)")
print("="*80)


DEMO: Single-Checkpoint Caching with Converse API

🔍 Query 1: What is the return policy for electronics?

💬 Response: According to the product manual, **electronics have a 14-day return window** from the purchase date....

Request 1
Input tokens:       15
Output tokens:      130
Cache write tokens: 1,936
Cache read tokens:  0


🔍 Query 2: How much does the Professional tier cost?

💬 Response: According to the product manual, the **Professional tier costs $299/month**.

This tier includes:
- ...

Request 2
Input tokens:       15
Output tokens:      88
Cache write tokens: 0
Cache read tokens:  1,936


🔍 Query 3: What shipping options are available?

💬 Response: Based on the product manual, the following shipping options are available:

1. **Standard Shipping**...

Request 3
Input tokens:       13
Output tokens:      182
Cache write tokens: 0
Cache read tokens:  1,936


🔍 Query 4: What are the API rate limits for standard tier?

💬 Response: According to the product manual, the API rate l

### Demo: Invoke Model API Example

Now let's run the same queries using the Invoke Model API to see that both APIs produce identical results.

In [7]:
# Same queries, using Invoke Model API
all_metrics_invoke = []

print("\n" + "="*80)
print("DEMO: Single-Checkpoint Caching with Invoke Model API")
print("="*80)

for i, query in enumerate(queries, 1):
    print(f"\n🔍 Query {i}: {query}")
    
    response, metrics = query_document_invoke(query)
    all_metrics_invoke.append(metrics)
    
    print(f"\n💬 Response: {response[:100]}..." if len(response) > 100 else f"\n💬 Response: {response}")
    print_cache_metrics(metrics, request_num=i)

# Calculate savings
print("\n" + "="*80)
print("COST ANALYSIS - Invoke Model API")
print("="*80)

savings_invoke = calculate_cache_savings(all_metrics_invoke)
print(f"\nTotal requests: {savings_invoke['total_requests']}")
print(f"Cache hit rate: {savings_invoke['cache_hit_rate']:.1f}%")
print(f"\nCost with caching:    ${savings_invoke['cost_with_cache']:.6f}")
print(f"Cost without caching: ${savings_invoke['cost_no_cache']:.6f}")
print(f"\n💰 Savings: ${savings_invoke['savings']:.6f} ({savings_invoke['savings_pct']:.1f}% reduction)")
print("="*80)

print("\n\n" + "="*80)
print("📊 COMPARISON: Converse vs Invoke Model")
print("="*80)
print(f"Both APIs achieve identical caching performance:")
print(f"  - Converse API cache hit rate: {savings['cache_hit_rate']:.1f}%")
print(f"  - Invoke Model API cache hit rate:   {savings_invoke['cache_hit_rate']:.1f}%")
print(f"  - Both APIs produce the same caching behavior!")
print("="*80)


DEMO: Single-Checkpoint Caching with Invoke Model API

🔍 Query 1: What is the return policy for electronics?

💬 Response: Based on the product manual, the return policy for electronics is:

**Electronics have a 14-day retu...

Request 1
Input tokens:       15
Output tokens:      145
Cache write tokens: 1,941
Cache read tokens:  0


🔍 Query 2: How much does the Professional tier cost?

💬 Response: Based on the product manual, the **Professional tier costs $299/month**.

This tier includes:
- Up t...

Request 2
Input tokens:       15
Output tokens:      88
Cache write tokens: 0
Cache read tokens:  1,941


🔍 Query 3: What shipping options are available?

💬 Response: Based on the product manual, the following shipping options are available:

1. **Standard Shipping**...

Request 3
Input tokens:       13
Output tokens:      182
Cache write tokens: 0
Cache read tokens:  1,941


🔍 Query 4: What are the API rate limits for standard tier?

💬 Response: Based on the product manual, the API rate l

### Key Observations from Section 1

From the demos above, you should see:

1. **Request 1**: Cache write tokens = document size (~2,000 tokens)
   - 📝 First occurrence, cache write (investment)
   - Higher cost: 1.25x regular input

2. **Requests 2-4**: Cache read tokens = document size (~2,000 tokens)
   - ✅ Cache hit! Content retrieved from cache
   - Lower cost: 0.1x regular input (~90% savings)

3. **Savings**: Typically 60-80% cost reduction with 4 requests

4. **API Differences**: Both Converse and Invoke Model APIs achieve identical caching results - only the syntax differs:
   - Converse: `{"cachePoint": {"type": "default"}}`
   - Invoke Model: `{"cache_control": {"type": "ephemeral"}}`

---

## Section 2: Multi-Checkpoint Caching

In Section 1, we used a **single cache checkpoint** to separate static content (document) from dynamic content (user queries). 

For more complex workflows, Amazon Bedrock supports **up to 4 cache checkpoints per request**, allowing you to cache different layers independently based on how frequently they change.

Cache checkpoints can be placed in:
- **Tools**: Tool definitions (in toolConfig)
- **System**: System prompts
- **Messages**: Conversation history

**Why multiple checkpoints?**
- Different components have different update frequencies
- Cache only what's stable, reprocess what changes
- Maximum cost efficiency for complex agentic workflows

**Key Points**:
- **Token minimums vary by model**: Check [AWS Bedrock Prompt Caching Docs](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-caching.html) for latest requirements
  - Example: Claude Sonnet 4.5 = 1,024 tokens, Claude Haiku 4.5 = 4,096 tokens
- Each checkpoint is **explicit** - you must add cache tags where you want caching

### Use Case: Customer Support Agent with Tools
For the following section, we'll implement a customer support agent with different cache checkpoints at Tools, System Prompts and Conversation history.

---

In [8]:
# Load tool definitions from file
with open('data/tool_definitions.json', 'r') as f:
    TOOL_DEFINITIONS = json.load(f)

# Estimate token count for tools (character-based for better accuracy)
# Token estimation: ~4 chars per token for JSON
tool_json_str = json.dumps(TOOL_DEFINITIONS)
estimated_tool_tokens = len(tool_json_str) / 4

print(f"✅ Tool definitions loaded from file")
print(f"   - {len(TOOL_DEFINITIONS)} tools defined")
print(f"   - Estimated tokens: ~{int(estimated_tool_tokens)} tokens")

✅ Tool definitions loaded from file
   - 5 tools defined
   - Estimated tokens: ~2523 tokens


In [9]:
# System prompt
with open('data/system_prompt.txt', 'r') as f:
    SYSTEM_PROMPT = f.read()

print("✅ System prompt loaded from file")
print(f"   Length: {len(SYSTEM_PROMPT.split())} words (~{int(len(SYSTEM_PROMPT.split())*1.3)} tokens)")

✅ System prompt loaded from file
   Length: 938 words (~1219 tokens)


### Mock Tool Functions

For this demo, we'll create simple mock functions that simulate tool execution without actual API calls.

In [10]:
# Mock tool functions for demo
def search_knowledge_base(query):
    """Mock knowledge base search"""
    return {
        "results": f"Knowledge base results for: {query}",
        "source": "CloudCommerce Help Center"
    }

def lookup_order(order_id):
    """Mock order lookup"""
    return {
        "order_id": order_id,
        "status": "Shipped",
        "eta": "2025-01-25",
        "tracking": "1Z999AA10123456784"
    }

print("✅ Mock tool functions defined")

✅ Mock tool functions defined


### Agent Implementation with Multi-Checkpoint Caching

The agent function implements three cache checkpoints:
1. **Checkpoint 1**: After tools (in tools array) - ~2,500 tokens
2. **Checkpoint 2**: After system prompt - ~1,200 tokens
3. **Checkpoint 3**: At end of conversation history - grows with each turn

In [11]:
def run_agent_with_caching(user_message, conversation_history=None, model_id=MODEL_ID):
    """
    Run support agent with multi-checkpoint caching.
    
    Caching Strategy (3 checkpoints total):
    - Tools: Cache checkpoint after tools
    - System: Cache checkpoint after system prompt
    - History: Cache checkpoint at END of entire conversation history
    - Current message: Not cached (most dynamic)
    
    Args:
        user_message: Current user message (can be None if only tool results in history)
        conversation_history: List of previous messages (optional)
        model_id: Bedrock model ID
    
    Returns:
        tuple: (response_text, cache_metrics, tool_calls, assistant_msg)
    """
    if conversation_history is None:
        conversation_history = []
    
    # Build messages - STRIP any existing cache checkpoints first
    messages = []
    
    # Add conversation history WITHOUT cache checkpoints
    if len(conversation_history) > 0:
        for msg in conversation_history:
            # Strip cache checkpoints from historical messages
            clean_msg = {"role": msg["role"], "content": []}
            
            # Clean content blocks (remove cache checkpoints)
            if isinstance(msg["content"], list):
                for block in msg["content"]:
                    if "text" in block:
                        clean_msg["content"].append({"text": block["text"]})
                    elif "toolUse" in block:
                        clean_msg["content"].append({"toolUse": block["toolUse"]})
                    elif "toolResult" in block:
                        # Tool results have their own structure (toolUseId + content)
                        clean_msg["content"].append({"toolResult": block["toolResult"]})
                    elif "json" in block:
                        # Tool result content blocks
                        clean_msg["content"].append({"json": block["json"]})
                    elif "toolUseId" in block:
                        # This is a tool result block (not wrapped in toolResult)
                        clean_msg["content"].append(block)
                    # Skip cachePoint blocks
            else:
                # Simple string content
                clean_msg["content"] = [{"text": msg["content"]}]
            
            messages.append(clean_msg)
        
        # CHECKPOINT 3: Add ONE cache checkpoint at end of entire history
        messages[-1]["content"].append({"cachePoint": {"type": "default"}})
    
    # Add current user message ONLY if not empty/None
    if user_message:
        messages.append({
            "role": "user",
            "content": [{"text": user_message}]
        })
    
    # CHECKPOINT 2: System prompt with cache checkpoint
    system = [
        {"text": SYSTEM_PROMPT},
        {"cachePoint": {"type": "default"}}
    ]
    
    # CHECKPOINT 1: Tools with cache checkpoint
    tool_config = {
        "tools": TOOL_DEFINITIONS + [{"cachePoint": {"type": "default"}}],
        "toolChoice": {"auto": {}}
    }
    
    response = bedrock_runtime.converse(
        modelId=model_id,
        messages=messages,
        system=system,
        toolConfig=tool_config,
        inferenceConfig={
            "maxTokens": 1000,
            "temperature": 0.0
        }
    )
    
    # Extract metrics
    metrics = extract_cache_metrics(response)
    
    # Handle tool calls
    stop_reason = response.get('stopReason')
    
    if stop_reason == 'tool_use':
        tool_calls = []
        for content_block in response['output']['message']['content']:
            if 'toolUse' in content_block:
                tool_calls.append(content_block['toolUse'])
        return None, metrics, tool_calls, response['output']['message']
    else:
        response_text = response['output']['message']['content'][0]['text']
        return response_text, metrics, None, None


def run_agent_turn(query, conversation, all_metrics):
    """
    Helper function to run a single agent turn with tool handling.
    
    Args:
        query: User query string
        conversation: Conversation history (will be modified in place)
        all_metrics: Metrics list (will be modified in place)
    
    Returns:
        tuple: (response_text, api_calls_list)
        where each api_call dict contains: {
            "metrics": {...},
            "stop_reason": "tool_use" or "end_turn",
            "description": "human-readable description"
        }
    """
    api_calls = []
    
    # API Call 1
    response, metrics, tool_calls, assistant_msg = run_agent_with_caching(query, conversation_history=conversation)
    all_metrics.append(metrics)
    
    # Record first API call
    stop_reason = "tool_use" if tool_calls else "end_turn"
    api_calls.append({
        "metrics": metrics,
        "stop_reason": stop_reason,
        "description": "Tool decision" if tool_calls else "Direct response"
    })
    
    # Handle tool calls if any
    if tool_calls:
        print(f"\n🔧 Agent wants to use tools:")
        
        # Build tool results
        tool_results = []
        for tool_call in tool_calls:
            tool_name = tool_call['name']
            tool_input = tool_call['input']
            print(f"   - {tool_name}({tool_input})")
            
            # Execute tool (mock)
            if tool_name == "search_knowledge_base":
                result = search_knowledge_base(tool_input.get('query', ''))
            elif tool_name == "lookup_order":
                result = lookup_order(tool_input.get('order_id', ''))
            
            print(f"   → Result: {result}")
        
            tool_results.append({
                "toolResult": {
                    "toolUseId": tool_call['toolUseId'],
                    "content": [{"json": result}]
                }
            })
        
        # Add to conversation
        conversation.append({"role": "user", "content": [{"text": query}]})
        conversation.append({"role": "assistant", "content": assistant_msg['content']})
        conversation.append({"role": "user", "content": tool_results})
        
        # API Call 2 - get final response
        response, final_metrics, _, _ = run_agent_with_caching(None, conversation_history=conversation)
        all_metrics.append(final_metrics)
        
        # Record second API call
        api_calls.append({
            "metrics": final_metrics,
            "stop_reason": "end_turn",
            "description": "Final response after tool execution"
        })
        
        # Add final response
        conversation.append({"role": "assistant", "content": [{"text": response}]})
    else:
        # No tools, just add query and response
        conversation.append({"role": "user", "content": [{"text": query}]})
        conversation.append({"role": "assistant", "content": [{"text": response}]})
    
    return response, api_calls


print("✅ Multi-checkpoint agent function defined")
print("✅ Helper function for agent turns defined")

✅ Multi-checkpoint agent function defined
✅ Helper function for agent turns defined


### Demo: Multi-Turn Conversation with Tools

**Instructions**: Run each cell below sequentially to see how caching evolves across conversation turns.

#### Turn 1: First Request (Cache Write)

In [12]:
# Initialize conversation tracking
conversation = []
all_metrics = []

# Turn 1: First user query
query = "I need help with my order ORD-12345"
print(f"{'='*80}")
print(f"Turn 1")
print(f"{'='*80}")
print(f"\n👤 User: {query}")

# Run agent turn (handles tools automatically)
response, api_calls = run_agent_turn(query, conversation, all_metrics)

print(f"\n🤖 Agent: {response}")

# Print cache metrics for all API calls in this turn
print(f"\n--- Cache Metrics for Turn 1 ---")
for i, call in enumerate(api_calls, 1):
    call_number = len(all_metrics) - len(api_calls) + i
    print(f"\n📊 API Call #{call_number} ({call['description']}):")
    print_cache_metrics(call['metrics'], request_num=call_number)

print(f"\n💡 Turn 1 Summary: Made {len(api_calls)} API call(s) total")

Turn 1

👤 User: I need help with my order ORD-12345

🔧 Agent wants to use tools:
   - lookup_order({'order_id': 'ORD-12345', 'include_history': True})
   → Result: {'order_id': 'ORD-12345', 'status': 'Shipped', 'eta': '2025-01-25', 'tracking': '1Z999AA10123456784'}

🤖 Agent: Great news! I found your order details:

**Order Status:** Shipped ✓

**Tracking Information:**
- Tracking Number: 1Z999AA10123456784
- Estimated Delivery: January 25, 2025

Your order is currently on its way! You can track your package using the tracking number above with the shipping carrier.

What specific help do you need with this order? For example:
- Need more details about the delivery?
- Want to make changes or cancel?
- Have questions about the items ordered?
- Need to initiate a return?

I'm here to assist!

--- Cache Metrics for Turn 1 ---

📊 API Call #1 (Tool decision):

Request 1
Input tokens:       334
Output tokens:      106
Cache write tokens: 4,052
Cache read tokens:  0


📊 API Call #2 (Final resp

**💡 What happened in Turn 1?**

This is the **"cold start"** - the initial investment in caching:

**API Call #1** (Tool decision):
- 📝 **Cache WRITE**: Tools + System = ~4,000 tokens cached
- 🆕 **Fresh processing**: User message "I need help with my order..."
- 💰 **Cost**: 1.25x regular price for cache write

**API Call #2** (Final response after tool execution):
- ✅ **Cache HIT**: Tools + System (already cached in API Call #1 - instant reuse)
- 📝 **Cache WRITE**: Turn 1 conversation with tool results (new content)
- 🆕 **Fresh processing**: Nothing (only processing tool results)

#### Turn 2: Cache Hit for Tools + System

In [13]:
query = "When will it arrive?"
print(f"{'='*80}")
print(f"Turn 2")
print(f"{'='*80}")
print(f"\n👤 User: {query}")

# Track how many API calls before this turn
calls_before = len(all_metrics)

# Run agent turn (handles tools automatically)
response, api_calls = run_agent_turn(query, conversation, all_metrics)

print(f"\n🤖 Agent: {response}")

# Print cache metrics for all API calls in this turn
print(f"\n--- Cache Metrics for Turn 2 ---")
for i, call in enumerate(api_calls, 1):
    call_number = calls_before + i
    print(f"\n📊 API Call #{call_number} ({call['description']}):")
    print_cache_metrics(call['metrics'], request_num=call_number)

print(f"\n💡 Turn 2 Summary: Made {len(api_calls)} API call(s) this turn ({len(all_metrics)} total so far)")

Turn 2

👤 User: When will it arrive?

🤖 Agent: Based on the tracking information for order ORD-12345, your package is estimated to arrive on **January 25, 2025**.

Since your order has already shipped, you can track its real-time progress using tracking number **1Z999AA10123456784** on the carrier's website for the most up-to-date delivery information.

Is there anything else you'd like to know about your order?

--- Cache Metrics for Turn 2 ---

📊 API Call #3 (Direct response):

Request 3
Input tokens:       10
Output tokens:      92
Cache write tokens: 141
Cache read tokens:  4,536


💡 Turn 2 Summary: Made 1 API call(s) this turn (3 total so far)


**💡 What happened in Turn 2?**

Cache behavior you'll see in the metrics above:
- ✅ **Cache HIT**: Tools + System (reused from Turn 1)
- 📝 **Cache WRITE**: Turn 1's conversation (first time seeing it, so cache it now)
- 🆕 **Fresh processing**: Current message "When will it arrive?"

**Why?** Tools and System stay the same across turns, so Bedrock reuses the cached version. Turn 1's conversation is new to the cache, so it gets written.

In [14]:
query = "Can you search the knowledge base about product warranty coverage?"
print(f"{'='*80}")
print(f"Turn 3")
print(f"{'='*80}")
print(f"\n👤 User: {query}")

# Track how many API calls before this turn
calls_before = len(all_metrics)

# Run agent turn (handles tools automatically)
response, api_calls = run_agent_turn(query, conversation, all_metrics)

print(f"\n🤖 Agent: {response}")

# Print cache metrics for all API calls in this turn
print(f"\n--- Cache Metrics for Turn 3 ---")
for i, call in enumerate(api_calls, 1):
    call_number = calls_before + i
    print(f"\n📊 API Call #{call_number} ({call['description']}):")
    print_cache_metrics(call['metrics'], request_num=call_number)

print(f"\n💡 Turn 3 Summary: Made {len(api_calls)} API call(s) this turn ({len(all_metrics)} total so far)")

Turn 3

👤 User: Can you search the knowledge base about product warranty coverage?

🔧 Agent wants to use tools:
   - search_knowledge_base({'query': 'product warranty coverage', 'category': 'products'})
   → Result: {'results': 'Knowledge base results for: product warranty coverage', 'source': 'CloudCommerce Help Center'}

🤖 Agent: I've searched our knowledge base for product warranty coverage information. Here's what I found:

**Product Warranty Coverage:**

Our warranty policies vary by product category and manufacturer. Generally:

- **Manufacturer's Warranty:** Most products come with the manufacturer's standard warranty (typically 1 year for electronics, varies by brand)
- **Extended Warranty:** Available for purchase at checkout for eligible items
- **Defective Items:** Covered regardless of warranty status - full refund or replacement with no time limit
- **Warranty Claims:** Can be processed through CloudCommerce or directly with the manufacturer

For specific warranty details 

**💡 What happened in Turn 3?**

Cache behavior you'll see in the metrics above:
- ✅ **Cache HIT**: Tools + System + Turn 1 conversation (all reused - maximum cache efficiency)
- 📝 **Cache WRITE**: Turn 2's conversation (first time seeing it, so cache it now)
- 🆕 **Fresh processing**: Current message about warranty

In [15]:
# Calculate savings across all turns
print("="*80)
print("COST ANALYSIS - Multi-Checkpoint Caching")
print("="*80)

savings = calculate_cache_savings(all_metrics)
print(f"\nTotal API calls: {savings['total_requests']}")
print(f"Cache hit rate: {savings['cache_hit_rate']:.1f}%")
print(f"\nCost with caching:    ${savings['cost_with_cache']:.6f}")
print(f"Cost without caching: ${savings['cost_no_cache']:.6f}")
print(f"\n💰 Savings: ${savings['savings']:.6f} ({savings['savings_pct']:.1f}% reduction)")
print("="*80)

COST ANALYSIS - Multi-Checkpoint Caching

Total API calls: 5
Cache hit rate: 78.7%

Cost with caching:    $0.024879
Cost without caching: $0.069921

💰 Savings: $0.045042 (64.4% reduction)


### Key Observations from Section 2

**Cache behavior across conversation turns**: As the conversation grows, MORE content gets cached and reused. This is why cache hit rate increases over time.

**The key insight**: 
- Turn 1: Cache everything for first time (higher investment)
- Turn 2: Reuse 2 layers (Tools + System)
- Turn 3: Reuse 3 layers (Tools + System + Turn 1)
- Turn N: Keep reusing more → even higher savings!

**When to use multi-checkpoint caching**:
- Production agentic workflows with tools
- Different layers have different update frequencies
- Long conversations where history accumulates
- Maximum control over what gets cached

---

## Section 3: Cache Invalidation

Now that you've seen multi-checkpoint caching in action, while caching is very useful in reducing both cost and latency, it's important to understand how Bedrock cache invalidation works to design the right caching strategy and maximize the benefits.

### Bedrock Prompt Assembly Order

When Bedrock assembles your prompt for inference, it follows this **strict sequential order**:

```
┌─────────────┐      ┌─────────────┐      ┌─────────────┐
│   Tools     │  →   │   System    │  →   │  Messages   │
│(if provided)│      │(if provided)│      │  (history + │
│             │      │             │      │   current)  │
└─────────────┘      └─────────────┘      └─────────────┘
```

This sequential assembly is the foundation of cache invalidation behavior. Bedrock uses **prefix matching** to determine cache hits:
- Caches are matched from the **beginning** of the assembled prompt
- When you add a cache checkpoint, Bedrock caches **everything from the start up to that checkpoint** - not just the individual section (cumulative caching)
- If content at any position changes, **that cache + all subsequent caches** are invalidated
- Only preceding caches (earlier in the sequence) remain valid

**Note**: You can place **multiple cache checkpoints** within any section (Tools, System, or Messages), with a maximum of **4 checkpoints per request**. For example, in Messages, you can have multiple cache points across different turns of conversation history.

### Visual Examples

**Example 1: Only message changes** ✅
```
Request 1: [Tools✓] → [System✓] → [Message: "Hello"]
           Cache₁     Cache₂      (no cache)
Request 2: [Tools✓] → [System✓] → [Message: "Help me"]
Result:    HIT✅     HIT✅        Fresh (not cached)
```
**Cache placement**: Cache₁ after Tools, Cache₂ after System

**Outcome**: Tools and System cache hits because they're unchanged and come before messages!

---

**Example 2: System changes** ⚠️
```
Request 1: [Tools✓] → [System: "Be helpful"✓] → [Message]
           Cache₁     Cache₂                    (no cache)
Request 2: [Tools✓] → [System: "Be concise"✗] → [Message]
Result:    HIT✅     MISS❌                       MISS❌
```
**Cache placement**: Cache₁ after Tools, Cache₂ after System

**Outcome**: 
- Cache₁ HIT (unchanged, comes first)
- Cache₂ MISS (content changed)
- Any downstream caches also invalidated (even if message was the same!)

---

**Example 3: Tools change** ❌❌
```
Request 1: [Tools: A, B✓] → [System✓] → [Message]
           Cache₁         Cache₂      (no cache)
Request 2: [Tools: A, B, C✗] → [System✓] → [Message]
Result:    MISS❌              MISS❌      MISS❌
```
**Cache placement**: Cache₁ after Tools, Cache₂ after System

**Outcome**: Everything invalidated! Tools are first in assembly order, so changing them breaks the entire cache chain.

---

**Example 4: Multiple cache points in Messages** 📚
```
Request 1: [Tools✓] → [System✓] → [Msg₁✓, Msg₂✓, Msg₃✓]
           Cache₁     Cache₂      Cache₃ Cache₄  (no cache)
Request 2: [Tools✓] → [System✓] → [Msg₁✓, Msg₂✓, Msg₃✓, Msg₄]
Result:    HIT✅     HIT✅        HIT✅   HIT✅   Fresh
```
**Cache placement**: Cache₁ after Tools, Cache₂ after System, Cache₃ after Msg₁, Cache₄ after Msg₂ (4 checkpoints max)

**Outcome**: Multiple cache points in the Messages section allow incremental caching of conversation history.

---

Let's see these scenarios in action with a data analyst agent example.

In [16]:
# Load data analyst tools and system
with open('data/analyst_tools.json', 'r') as f:
    ANALYST_TOOLS = json.load(f)

with open('data/analyst_system.txt', 'r') as f:
    ANALYST_SYSTEM_BASE = f.read()

# Create two versions with different priorities
ANALYST_SYSTEM_V1 = ANALYST_SYSTEM_BASE + "\n\nPRIORITY: Focus on accuracy and thoroughness."
ANALYST_SYSTEM_V2 = ANALYST_SYSTEM_BASE + "\n\nPRIORITY: Focus on speed and quick insights."

# Calculate estimated token counts
tool_json_str = json.dumps(ANALYST_TOOLS)
estimated_tool_tokens = len(tool_json_str) / 4
estimated_system_tokens = len(ANALYST_SYSTEM_V1.split()) * 1.3

print("✅ Data analyst scenario loaded")
print(f"   Tools: {len(ANALYST_TOOLS)} tools (~{int(estimated_tool_tokens)} tokens)")
print(f"   System V1: ~{len(ANALYST_SYSTEM_V1.split())} words (~{int(estimated_system_tokens)} tokens)")
print(f"   System V2: ~{len(ANALYST_SYSTEM_V2.split())} words (~{int(estimated_system_tokens)} tokens)")

# Helper function for analyst demos
def run_analyst_demo(user_message, system_text, tools_list, marker=""):
    """Run analyst agent with specified tools and system"""
    messages = [{
        "role": "user",
        "content": [{"text": user_message + marker}]  # Marker for unique caches
    }]
    
    system = [
        {"text": system_text},
        {"cachePoint": {"type": "default"}}
    ]
    
    tool_config = {
        "tools": tools_list + [{"cachePoint": {"type": "default"}}],
        "toolChoice": {"auto": {}}
    }
    
    response = bedrock_runtime.converse(
        modelId=MODEL_ID,
        messages=messages,
        system=system,
        toolConfig=tool_config,
        inferenceConfig={"maxTokens": 500, "temperature": 0.0}
    )
    
    return extract_cache_metrics(response)

✅ Data analyst scenario loaded
   Tools: 2 tools (~1182 tokens)
   System V1: ~566 words (~735 tokens)
   System V2: ~567 words (~735 tokens)


### ⚠️ Important: Cumulative Token Counting

**Notice something interesting?**
- Tools = ~1,200 tokens ✅ (meets 1,024 minimum)
- System = ~750 tokens ❌ (below 1,024 minimum)

**Question**: How can System be cached if it's only 750 tokens?

**Answer**: Remember cumulative caching from earlier? Bedrock counts tokens **from the start** to each checkpoint, not individually.

**Checkpoint 1** (after Tools):
```
[Tools: 1,200 tokens] → CHECKPOINT
Total: 1,200 tokens ✅ → CACHED
```

**Checkpoint 2** (after System):
```
[Tools: 1,200] + [System: 750] → CHECKPOINT
Total: 1,950 tokens ✅ → CACHED (Tools + System together)
```

**Key insight**: Even if System alone is below 1,024 tokens, the **cumulative total from the beginning (1,950 tokens)** meets the minimum, so Checkpoint 2 caches the entire Tools + System prefix.

---

Let's test with our 1st scenario by changing the system prompts.

In [17]:
print("\n" + "="*80)
print("SCENARIO 1: System Prompt Changes")
print("="*80)
print("Demonstrating: Tools stay same, System changes → Tools HIT, System MISS")
print("="*80)

# Request 1: Analyst with "accuracy priority"
metrics1 = run_analyst_demo(
    user_message="Show sales trends", 
    system_text=ANALYST_SYSTEM_V1,
    tools_list=ANALYST_TOOLS,
    marker=" [Scenario1]"
)

print("\n📊 Request 1: System = 'Focus on accuracy'")
print_cache_metrics(metrics1)

# Request 2: Analyst with "speed priority" (System changed!)
metrics2 = run_analyst_demo(
    user_message="Show sales trends",
    system_text=ANALYST_SYSTEM_V2,  # DIFFERENT SYSTEM!
    tools_list=ANALYST_TOOLS,  # Same tools
    marker=" [Scenario1]"
)

print("\n📊 Request 2: System = 'Focus on speed' (CHANGED!)")
print_cache_metrics(metrics2)


SCENARIO 1: System Prompt Changes
Demonstrating: Tools stay same, System changes → Tools HIT, System MISS

📊 Request 1: System = 'Focus on accuracy'

Cache Metrics
Input tokens:       331
Output tokens:      75
Cache write tokens: 1,946
Cache read tokens:  0


📊 Request 2: System = 'Focus on speed' (CHANGED!)

Cache Metrics
Input tokens:       331
Output tokens:      75
Cache write tokens: 753
Cache read tokens:  1,193



**💡 What happened in Scenario 1?**

**Request 1** (System prompt = '...Focus on accuracy'):
- 📝 **Cache WRITE**: Tools (~1,200 tokens) + System V1 (~750 tokens) = ~2,000 tokens cached
- 🆕 **Fresh processing**: User message "Show sales trends [Scenario1]"
- 💰 **Cost**: 1.25x for cache write (initial investment)

**Request 2** (System prompt = '...Focus on speed' - CHANGED!):
- ✅ **Cache HIT**: Tools (~1,200 tokens) - same tools, reused from Request 1
- 📝 **Cache WRITE**: System V2 (~750 tokens) - different priority, new cache
- 🆕 **Fresh processing**: Same user message

**Key insight**: 
- Tools come FIRST in assembly order → Tools cache survives system change ✅
- Changing System only invalidates System cache and beyond, NOT Tools
- Partial cache reuse saves cost even when System changes

Let's test with our 2st scenario by changing the tools definition.

In [18]:
print("\n" + "="*80)
print("SCENARIO 2: Tools Change")
print("="*80)
print("Demonstrating: Tools change → Everything invalidated (Tools + System)")
print("="*80)

# Request 1: Continue from Scenario 1's end state (same Tools + System V2)
# This should show FULL cache hit since both were already cached
metrics1 = run_analyst_demo(
    user_message="Analyze data",
    system_text=ANALYST_SYSTEM_V2,  # Same System V2 from end of Scenario 1
    tools_list=ANALYST_TOOLS,  # Same tools: query_database, generate_chart
    marker=" [Scenario2]"  # Different marker to separate from Scenario 1
)

print("\n📊 Request 1: Same Tools + System V2 (from Scenario 1)")
print_cache_metrics(metrics1)

# Create a new tool (export_report)
NEW_ANALYST_TOOL = {
    "toolSpec": {
        "name": "export_report",
        "description": "Export analysis report to PDF or Excel format",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "format": {"type": "string", "enum": ["pdf", "excel"]},
                    "data": {"type": "object", "description": "Report data"}
                },
                "required": ["format", "data"]
            }
        }
    }
}

# Request 2: Add third tool (Tools changed!)
ANALYST_TOOLS_MODIFIED = [NEW_ANALYST_TOOL] + ANALYST_TOOLS
metrics2 = run_analyst_demo(
    user_message="Analyze data",
    system_text=ANALYST_SYSTEM_V2,  # Same System V2
    tools_list=ANALYST_TOOLS_MODIFIED,  # 3 tools now (CHANGED!)
    marker=" [Scenario2]"
)

print("\n📊 Request 2: Tools = [query_database, generate_chart, export_report] (CHANGED!)")
print_cache_metrics(metrics2)


SCENARIO 2: Tools Change
Demonstrating: Tools change → Everything invalidated (Tools + System)

📊 Request 1: Same Tools + System V2 (from Scenario 1)

Cache Metrics
Input tokens:       331
Output tokens:      192
Cache write tokens: 0
Cache read tokens:  1,946


📊 Request 2: Tools = [query_database, generate_chart, export_report] (CHANGED!)

Cache Metrics
Input tokens:       331
Output tokens:      195
Cache write tokens: 2,031
Cache read tokens:  0



**💡 What happened in Scenario 2?**

**Request 1** (Same Tools + System V2 from Scenario 1):
- ✅ **Cache HIT**: Tools (~1,200 tokens) & System V2 (~750 tokens) - already cached in Scenario 1
- 💰 **Cost**: Only 0.1x for cache reads - maximum savings!

**Request 2** (Added third tool - CHANGED!):
- ❌ **Cache MISS**: Tools (~1,300 tokens) - content changed (3 tools instead of 2)
- ❌ **Cache MISS**: System V2 (~750 tokens) - invalidated due to Tools change
- 📝 **Cache WRITE**: New Tools (3 tools) + System V2 written to cache
- 💰 **Cost**: 1.25x for cache write - expensive restart!

**Key insight**: 
- Tools come FIRST → Changing tools breaks ENTIRE cache chain
- Even though System V2 was unchanged, it got invalidated because Tools (which come before it) changed
- Adding/removing/modifying tools is the most expensive cache invalidation

(Continue with Part B)